In [2]:
# Intallation and Imports

# OpenAI (v0.28), Pandas, and tqdm
!pip -q install "openai==0.28" pandas tqdm 

import os, json, gzip, time, difflib
import pandas as pd
from tqdm import tqdm
from typing import Dict, Any, List, Tuple
from getpass import getpass
import openai
import random
import copy

print("OpenAI SDK version:", openai.__version__)

OpenAI SDK version: 0.28.0


In [3]:
# API Key Request

openai.api_key = getpass("Enter your OpenAI API key (will not echo): ")

In [4]:
# Configuration

FILE_PATH         = "DATA/abcd_v1.1.json"   
ONTOLOGY_PATH     = "DATA/ontology.json" # optional; enrich label sets if present
PRIMARY_MODEL     = "gpt-4o" 
FALLBACK_MODELS   = ["gpt-4o", "gpt-3.5-turbo-0125"]
MAX_SAMPLES  = 250 # Cap for cost. Use 250 at max
REQUEST_DELAY_SEC = 0.7

In [5]:
# Helper Functions

def call_chat_model_safely(messages, model):
    try:
        resp = openai.ChatCompletion.create(model=model, messages=messages, temperature=0)
        return resp["choices"][0]["message"].get("content","") if resp.get("choices") else ""
    except Exception as e:
        print(f"[chat:{model}] error: {e}")
        return "" 

def random_seeded_examples(dataset, size=250, seed=42):
    random.seed(seed)
    return random.sample(dataset, size)

def load_json(path: str):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)
    
def convo_to_transcript(convo: Dict[str,Any]) -> str:
    orig = convo.get("original", [])
    return " ".join([f"{sp}: {tx}" for sp, tx in orig])

def try_parse_json(text: str):
    if not text: return None
    text = text.strip()
    try:
        return json.loads(text)
    except Exception:
        s, e = text.find("{"), text.rfind("}")
        if s != -1 and e != -1 and e > s:
            cand = text[s:e+1]
            try:
                return json.loads(cand)
            except Exception:
                return None
    return None

def closest_label(pred: str, choices: List[str], cutoff: float = 0.6) -> str:
    if not pred or not choices:
        return ""
    if pred in choices:
        return pred
    # Case-insensitive exact first
    lowmap = {c.lower(): c for c in choices}
    if pred.lower() in lowmap:
        return lowmap[pred.lower()]
    # Fuzzy match to nearest valid label
    best = difflib.get_close_matches(pred, choices, n=1, cutoff=cutoff)
    return best[0] if best else ""

In [6]:
# Load Dataset & Split into Train, Dev, and Test

abcd = load_json(FILE_PATH)
train_split = (abcd.get("train", []) or [])
dev_split = (abcd.get("dev", []) or [])
test_split = (abcd.get("test", []) or [])
print(f"Train examples: {len(train_split)} | Dev examples: {len(dev_split)} | Test examples: {len(test_split)}")

# Gather 250 random seeded examples of train_split, dev_split, and test_split
# Use for consistent sampling during prompt testing, validation, and final evaluation

train_seeded_examples = random_seeded_examples(train_split)
dev_seeded_examples = random_seeded_examples(dev_split)
test_seeded_examples = random_seeded_examples(test_split)
print(f"Seeded Train examples: {len(train_seeded_examples)} | Seeded Dev examples: {len(dev_seeded_examples)} | Seeded Test examples: {len(test_seeded_examples)}")

Train examples: 8034 | Dev examples: 1004 | Test examples: 1004
Seeded Train examples: 250 | Seeded Dev examples: 250 | Seeded Test examples: 250


In [ ]:
# Build label sets

SCHEMA = {
  "personal": {"customer_name":"","email":"","member_level":"","phone":"","username":""},
  "order": {"street_address":"","full_address":"","city":"","num_products":"","order_id":"",
            "packaging":"","payment_method":"","products":"[]","purchase_date":"","state":"","zip_code":""},
  "product": {"names":[],"amounts":[]},
  "flow": "",
  "subflow": ""
}

def extract_unique_values_from_conversations(dataset: List[Dict[str, Any]]) -> Dict[str, Any]:
    # Takes a list of dictionaries(the dataset) and returns one dictionary 
    # Easy access to unique values of each feature

    unique_values = {
        dictionary: (
            {subdictionary: set() for subdictionary in subdict_content}
            if isinstance(subdict_content, dict) else set()
        )
        for dictionary, subdict_content in SCHEMA.items() # Gathered the dictionary getters from SCHEMA
    }

    # Iterate through all conversations and collect values
    for example in dataset:
        scenario = example.get("scenario", {})

        for dictionary, subdict_content in SCHEMA.items():
            # Handle nested dictionaries (personal, order, product)
            if isinstance(subdict_content, dict): 
                subdict = scenario.get(dictionary, {})
                for subdictionary in subdict_content.keys():
                    value = subdict.get(subdictionary)
                    if value not in (None, ""):
                        # If value is a list, add each item individually
                        if isinstance(value, list):
                            unique_values[dictionary][subdictionary].update(value)
                        else:
                            unique_values[dictionary][subdictionary].add(value)
            # Handle non-nested dictionary (flow, subflow)
            else:
                value = scenario.get(dictionary)
                if value not in (None, ""):
                    unique_values[dictionary].add(value)

    # Ensure every set includes None    THIS IS NEW, CHECK BEFORE PUSH TO MAIN
    for dictionary, subdict_content in unique_values.items():
        if isinstance(subdict_content, dict):
            for subdictionary in subdict_content:
                subdict_content[subdictionary].add('None')
        else:
            subdict_content.add('None')

    return unique_values

label_sets = extract_unique_values_from_conversations(train_split)

# def enrich_with_ontology(unique_values: Dict[str, Any], ontology_path: str) -> Dict[str, Any]:
#     try:
#         with open(ontology_path, "r", encoding="utf-8") as f:
#             ontology = json.load(f)

#         def walk_and_add(item, path=[]):
#             # If it's a dictionary, dive into each key-value pair
#             if isinstance(item, dict):
#                 for key, value in item.items():
#                     walk_and_add(value, path + [key])
#             # If it's a list, check each element
#             elif isinstance(item, list):
#                 for element in item:
#                     walk_and_add(element, path)
#             # If it's a string, add it to matching feature sets
#             elif isinstance(item, str):
#                 for section, content in unique_values.items():
#                     if isinstance(content, dict):
#                         for field_name in content.keys():
#                             if field_name in path or section in path:
#                                 unique_values[section][field_name].add(item)
#                     else:
#                         if section in path:
#                             unique_values[section].add(item)

#         walk_and_add(ontology)
#         print(f"[ontology] Successfully enriched features with values from {ontology_path}")

#     except Exception as e:
#         print(f"[ontology] Could not parse {ontology_path}: {e}")

#     return unique_values

# label_sets = enrich_with_ontology(label_sets, ONTOLOGY_PATH)
# print(label_sets['order']['num_products'])

{'None', '2', '1', '3'}


In [8]:
print(label_sets)

{'personal': {'customer_name': {'alessandro phoenix', 'david williams', 'rodriguez domingo', 'joyce wu', 'None', 'crystal minh', 'joseph banter', 'sanya afzal', 'albert sanders', 'norman bouchard', 'chloe zhang'}, 'email': {'jw742350@email.com', 'davidwilliams953@email.com', 'alessandrophoenix061@email.com', 'rodriguezd083@email.com', 'cminh837@email.com', 'joycew04@email.com', 'dw402198@email.com', 'sa128316@email.com', 'albertsanders030@email.com', 'dw407221@email.com', 'nbouchard973@email.com', 'sanyaafzal432@email.com', 'crystalminh120@email.com', 'jwu52873@email.com', 'joycew65@email.com', 'rodriguezd298@email.com', 'rdomingo203@email.com', 'aphoenix962@email.com', 'albertsanders470@email.com', 'chloez18@email.com', 'josephbanter961@email.com', 'albertsanders165@email.com', 'jwu95262@email.com', 'nbouchard165@email.com', 'alessandrop292@email.com', 'nb324373@email.com', 'crystalminh109@email.com', 'crystalm727@email.com', 'davidw14@email.com', 'cminh948@email.com', 'albertsanders1

In [ ]:
# Metadata extractor

# Full list of features. Only use the features you are responsible for
label_opts = {
"customer_name": f"- Valid customer_name labels (pick exactly one, copy verbatim): {label_sets['personal']['customer_name']}\n",
"email": f"- Valid email labels (pick exactly one, copy verbatim): {label_sets['personal']['email']}\n",
"memeber_level": f"- Valid member_level labels (pick exactly one, copy verbatim): {label_sets['personal']['member_level']}\n",
"phone": f"- Valid phone labels (pick exactly one, copy verbatim): {label_sets['personal']['phone']}\n",
"username": f"- Valid username labels (pick exactly one, copy verbatim): {label_sets['personal']['username']}\n",
"street_address": f"- Valid street_address labels (pick exactly one, copy verbatim): {label_sets['order']['street_address']}\n",
"full_address": f"- Valid full_address labels (pick exactly one, copy verbatim): {label_sets['order']['full_address']}\n",
"city": f"- Valid city labels (pick exactly one, copy verbatim): {label_sets['order']['city']}\n",
"num_products": (
    f"- Valid num_products labels (pick exactly one, copy verbatim): {label_sets['order']['num_products']}\n"
    "- **CORE TASK:** Determine the total number of **distinct, physical products or items** that are the central subject of the discussion (e.g., being ordered, refunded, questioned, or mentioned by name).\n"
    "\n"
    "--- **IMPORTANT: What NOT to Count** ---\n"
    "- **Do NOT count dollar amounts, refund IDs, or separate charges as products.** \n"
    "- **Example:** A discussion about an 'existing $100 refund' and adding a 'new $74 refund' for 'Guess jeans' counts as **'1'** product (the jeans), not '2'.\n"
    "\n"
    "--- **DECISION LOGIC (Follow in order)** ---\n"
    "\n"
    "**STEP 1: Check for 'None' (Zero Products)**\n"
    "   - **Select 'None' ONLY IF** the conversation is *purely* administrative and non-transactional (e.g., password reset, general policy question, 'what payment methods do you accept?').\n"
    "   - **FAILSAFE:** If the transcript mentions an Order ID, Tracking Number, Refund, or Cancellation, it is **NOT 'None'**. Proceed to Step 2.\n"
    "\n"
    "**STEP 2: Check for 'Multiple' Products ('2' or '3')**\n"
    "   - **Select '2' or '3' ONLY IF** there is clear, explicit evidence of more than one distinct item being discussed.\n"
    "   - **Look for:**\n"
    "     * **Explicit Lists:** 'the shirt and the shoes,' 'item A and item B.'\n"
    "     * **Separate Actions on Separate Items:** 'a refund for the hat AND tracking for the pants.'\n"
    "     * **Strong Contextual Plurals:** 'I have issues with **both items**,' 'neither of these **two products** work.' (Weak plurals like 'my items' are not enough).\n"
    "\n"
    "**STEP 3: Default to '1' (Single Product)**\n"
    "   - **If you did not select 'None' (Step 1) AND you did not find clear evidence for 'Multiple' (Step 2), then select '1'.**\n"
    "   - This is the default for any transaction-related discussion (Order ID, Refund, etc.) where the text refers to 'the item,' 'the product,' or 'the order' in the singular, or doesn't specify a number.\n"
),
"order_id": f"- Valid order_id labels (pick exactly one, copy verbatim): {label_sets['order']['order_id']}\n",
"packaging": f"- Valid packaging labels (pick exactly one, copy verbatim): {label_sets['order']['packaging']}\n",
"payment_method": f"- Valid payment_method labels (pick exactly one, copy verbatim): {label_sets['order']['payment_method']}\n",
"products": f"- Valid products labels (pick exactly one, copy verbatim): {label_sets['order']['products']}\n",
"purchase_date": f"- Valid purchase_date labels (pick exactly one, copy verbatim): {label_sets['order']['purchase_date']}\n",
# "state": f"- Valid state labels (pick exactly one, copy verbatim): {label_sets['order']['state']}\n",
"state": (
    f"- Valid state labels (pick exactly one, copy verbatim): {label_sets['order']['state']}\n"
    "- **CORE TASK:** Identify the **US state** associated with the customer's order, shipping address, or billing address that is **explicitly mentioned** in the conversation.\n"
    "\n"
    "--- **IMPORTANT: What TO Extract** ---\n"
    "- **Extract ONLY if the state is clearly stated in a complete address context** (e.g., shipping address, billing address, order details).\n"
    "- **Valid formats include:**\n"
    "  * **Full state names:** 'California', 'Texas', 'New York'\n"
    "  * **2-letter abbreviations:** 'CA', 'TX', 'NY' (in address context)\n"
    "  * **Within full addresses:** '123 Main St, Los Angeles, CA 90210' → 'ca'\n"
    "\n"
    "--- **IMPORTANT: What NOT to Extract** ---\n"
    "- **Do NOT extract states mentioned in non-address contexts:**\n"
    "  * Store locations: 'Our warehouse is in Texas' → **'None'** (unless it's the customer's state)\n"
    "  * General references: 'I'm visiting California' → **'None'** (unless ordering/shipping there)\n"
    "  * Agent location: 'I'm calling from our NY office' → **'None'**\n"
    "- **Do NOT extract ambiguous abbreviations** without address context:\n"
    "  * 'OK' (could mean 'okay' or 'Oklahoma') → **'None'** unless clearly in an address\n"
    "  * 'IN' (could mean 'in' or 'Indiana') → **'None'** unless clearly in an address\n"
    "  * 'OR' (could mean 'or' or 'Oregon') → **'None'** unless clearly in an address\n"
    "\n"
    "--- **DECISION LOGIC (Follow in order)** ---\n"
    "\n"
    "**STEP 1: Check for 'None' (No State Information)**\n"
    "   - **Select 'None' IF:**\n"
    "     * No address information is mentioned in the conversation\n"
    "     * The conversation is purely about products, refunds, or policies without location details\n"
    "     * State abbreviations appear but NOT in an address context (e.g., 'OK' meaning 'okay')\n"
    "     * Only international addresses are mentioned (non-US)\n"
    "\n"
    "**STEP 2: Look for Explicit Address Components**\n"
    "   - **Select the state abbreviation (lowercase) ONLY IF you find:**\n"
    "     * **Complete address:** 'My address is 456 Oak Ave, Austin, Texas 78701' → 'tx'\n"
    "     * **Partial address with state:** 'Ship to 789 Elm St, Seattle, WA' → 'wa'\n"
    "     * **State in order confirmation:** 'Order shipping to Miami, Florida' → 'fl'\n"
    "     * **Customer provides state explicitly:** 'I live in California' or 'My billing state is NY' → 'ca' or 'ny'\n"
    "   - **Key indicators:**\n"
    "     * Words like: 'address', 'ship to', 'billing', 'delivery', 'located at', 'zip code'\n"
    "     * City + State combinations: 'Boston, MA', 'Denver, Colorado'\n"
    "     * State + ZIP code: 'CA 90210', 'Texas 78701'\n"
    "\n"
    "**STEP 3: Normalize to 2-Letter Lowercase Abbreviation**\n"
    "   - **Convert full state names to lowercase abbreviations:**\n"
    "     * 'California' or 'CALIFORNIA' → 'ca'\n"
    "     * 'New York' → 'ny'\n"
    "     * 'North Carolina' → 'nc'\n"
    "   - **Convert uppercase abbreviations to lowercase:**\n"
    "     * 'TX' → 'tx'\n"
    "     * 'WA' → 'wa'\n"
    "\n"
    "**STEP 4: Verify Context (Critical)**\n"
    "   - **Before finalizing, confirm the state is:**\n"
    "     * Part of the customer's shipping/billing/order address\n"
    "     * NOT just mentioned casually or by the agent about their own location\n"
    "     * NOT an ambiguous word that could mean something else\n"
    "\n"
    "--- **EXAMPLES** ---\n"
    "1. 'My address is 123 Main St, Los Angeles, CA 90210' → **'ca'**\n"
    "2. 'Ship to 456 Elm, Austin, Texas' → **'tx'**\n"
    "3. 'I need to update my billing address to WA' → **'wa'**\n"
    "4. 'The item is out of stock in our Texas warehouse' → **'None'** (not customer's state)\n"
    "5. 'Customer says OK' → **'None'** (not Oklahoma, just agreement)\n"
    "6. 'I live in New York' → **'ny'** (customer's location)\n"
    "7. 'Can you check if it's in stock?' → **'None'** ('in' is not Indiana)\n"
),
"zip_code": f"- Valid zip_code labels (pick exactly one, copy verbatim): {label_sets['order']['zip_code']}\n",
"names": f"- Valid names labels (pick exactly one, copy verbatim): {label_sets['product']['names']}\n",
"amounts": f"- Valid amounts labels (pick exactly one, copy verbatim): {label_sets['product']['amounts']}\n",
"flow": f"- Valid flow labels (pick exactly one, copy verbatim): {label_sets['flow']}\n",
"subflow": f"- Valid subflow labels (pick exactly one, copy verbatim): {label_sets['subflow']}\n"    
}

def extract_metadata_from_transcript(transcript: str, opts: list, label_sets: dict=label_sets) -> dict:
    
    selected_instr = ''.join(label_opts[opt] for opt in opts if opt in label_opts) # Join selected label instructions
    label_instr = (
    "CLASSIFICATION CONSTRAINTS:\n"
    + selected_instr +
    "- Do NOT invent new labels. If uncertain, pick the most likely from the lists.\n"
    )

    # Build prompt
    prompt = (
        "Convert the customer-support dialog into structured metadata.\n\n"
        f"{label_instr}\n"
        "OUTPUT RULES:\n"
        "- Return STRICT JSON only (no prose, no markdown).\n"
        "- Use this exact schema and field types:\n"
        f"{json.dumps(SCHEMA, separators=(',', ':'))}\n" # Removed ", indent=2" because it increases tokenization. Only useful for readability/debugging
        "- If a field is missing, use \"\" or [] accordingly.\n\n"
        f"Dialog transcript:\n{transcript}\n"
    )

    messages = [
        {"role": "system", "content": "Always return valid JSON that exactly matches the schema. No explanations."},
        {"role": "user", "content": prompt}
    ]

    # Try primary and fallback models
    models_to_try = [PRIMARY_MODEL] + [m for m in FALLBACK_MODELS if m != PRIMARY_MODEL]

    for model_name in models_to_try:
        content = call_chat_model_safely(messages, model_name)
        data = try_parse_json(content)
        if isinstance(data, dict):
            # Deep copy schema to ensure fresh structure
            out = copy.deepcopy(SCHEMA)
            # Populate with model output
            for key, value in data.items():
                out[key] = value

        # Post-process: normalize labels using closest_label()
            for key, value in out.items():
                if key in label_sets and isinstance(value, str):
                    out[key] = closest_label(value, label_sets[key])
                elif key in label_sets and isinstance(value, list):
                    out[key] = [closest_label(v, label_sets[key]) for v in value]

            return out
        
        if content:
            print(f"[warn:{model_name}] unparsable output (first 160 chars): {content[:160]}")
        time.sleep(REQUEST_DELAY_SEC)

    # If no model produced valid JSON, return empty schema
    return copy.deepcopy(SCHEMA)

In [10]:
# Keep excess or experimental prompt/extraction code here (not executed). CLEAR EVERYTHING BELOW THIS LINE BEFORE PUSHING TO MAIN!!!

In [48]:
# Essential prompt testing setup functions

# Dataframe creator
def build_dataframe(examples: list, features_to_include: list, max_samples: int=MAX_SAMPLES):
    rows = []
    features_to_include = set(features_to_include) if features_to_include else None # A set looks up the dictionary keys faster than a list does

    for example in examples[:max_samples]:

        scenario = example.get("scenario", {})
        # Flatten scenario keys
        personal = scenario.get("personal", {})
        order = scenario.get("order", {})
        product = scenario.get("product", {})

        features = {
            "customer_name": personal.get("customer_name"),
            "email": personal.get("email"),
            "member_level": personal.get("member_level"),
            "phone": personal.get("phone"),
            "username": personal.get("username"),
            "street_address": order.get("street_address"),
            "full_address": order.get("full_address"),
            "city": order.get("city"),
            "num_products": order.get("num_products"),
            "order_id": order.get("order_id"),
            "packaging": order.get("packaging"),
            "payment_method": order.get("payment_method"),
            "products": order.get("products"),
            "purchase_date": order.get("purchase_date"),
            "state": order.get("state"),
            "zip_code": order.get("zip_code"),
            "names": product.get("names"),
            "amounts": product.get("amounts"),
            "flow": scenario.get("flow"),
            "subflow": scenario.get("subflow"),
        }

        # This code keeps only the features you want in the features dictionary
        if features_to_include:
            features = {key: value for key, value in features.items() if key in features_to_include}
        rows.append({
            "convo_id": example.get("convo_id",""),
            **features,
            "transcript": convo_to_transcript(example),
        })
    
    return pd.DataFrame(rows)

# Make predictions (extraction)
def extraction(df, feature_options : list):
    predicted_feature_values = []
    for transcript in tqdm(df["transcript"], desc="Extracting Metadata"):
        predicted_feature_values.append(extract_metadata_from_transcript(transcript, feature_options))
        time.sleep(REQUEST_DELAY_SEC)  
    
    # Makes feature names appear cleanly (example: "email" instead of "personal.email")
    extracted = []
    for section in ["personal", "order", "product", "flow", "subflow"]:
        if section in ["flow", "subflow"]:
            extract = pd.DataFrame({f"extracted_{section}": [item[section] for item in predicted_feature_values]})
        else:
            extract = pd.json_normalize([item[section] for item in predicted_feature_values]).add_prefix("extracted_") 
        extracted.append(extract)
    extracted_df = pd.concat(extracted, axis=1).reset_index(drop=True)

    # Makes column values lowercase THIS IS NEW
    # for col in extracted_df:
    #     exampl = 0
    #     for val in extracted_df[col]:
    #         extracted_df.loc[exampl, col] = val.lower()
    #         exampl+=1
    extracted_df['extracted_member_level'] = extracted_df['extracted_member_level'].str.lower()
    ###End of new stuff, modify before pushing to github
    
    final_df = pd.concat([df.reset_index(drop=True), extracted_df[[f"extracted_{f}" for f in feature_options]]], axis=1)

    return final_df

# Measures extraction accuracy per feature
def accuracy(df, feature_options: list):
    for feature in feature_options:
        gt = df[feature].astype(str).fillna("")
        ex = df[f"extracted_{feature}"].astype(str).fillna("")
        acc = (gt == ex).mean()
        print(f"{feature} accuracy: {acc: .2%}")

In [49]:
# ============================================================
# 1️. TRAINING EVALUATION
# ============================================================
# Purpose: Test your extraction function (extract_metadata_from_transcript)
#          on training examples to refine prompt behavior and schema.
# Notes:
# - Run this cell often while tuning prompts.
# - Expect to iterate and modify the extraction prompt/function here.
# ============================================================

feature_options = ["num_products"] # Include the features you are supposed to test (ex. ["flow", "subflow"]). CLEAR VALUE BEFORE PUSHING FILE TO MAIN!!!
examples = 68 # Can use variable as 3rd arugment for build_dataframe (default is 250). You do not have to rerun prior code cells

df = build_dataframe(train_seeded_examples, feature_options, examples)
print("DataFrame shape:", df.shape)

final_df = extraction(df, feature_options)
pd.set_option("display.max_colwidth", 25)
display(final_df.head(250))

accuracy(final_df, feature_options)


DataFrame shape: (68, 3)


Extracting Metadata: 100%|██████████| 68/68 [02:40<00:00,  2.36s/it]


,convo_id,num_products,transcript,extracted_num_products
0,1746,1,"agent: Hello, how may...",1
1,9098,2,customer: hi customer...,1
2,1272,1,agent: Hello thanks f...,1
3,5506,1,customer: I would lik...,1
4,5375,1,"agent: Hello, how may...",1
...,...,...,...,...
63,5077,None,"agent: hello, how can...",None
64,6629,None,agent: Hi and welcome...,None
65,5833,3,"agent: Hello, how can...",1
66,4462,1,"agent: Hi, there! Ho...",1


num_products accuracy:  72.06%


In [46]:
for i in range(10):
    print(final_df['transcript'][i])

print(final_df['transcript'][65])

agent: Hello, how may I help you? customer: I want to know the state of my refund agent: Let me help you with that. customer: I have an existing refund of $100 + I want to refund another $74 agent: Did you want to add an extra item to your current refund? customer: Yes agent: Could I have your full name or account id? customer: Albert sanders customer: Account Id 123445 action: Account has been pulled up for Albert sanders. customer: Thanks agent: Could I have your username, email address and order id to validate your order? customer: alberts7 customer: alberts7@email.com agent: And the order id? customer: 0712999033 action: Purchase validation in progress ... agent: Thank you agent: What is the  item that you want to return? customer: Jeans customer: Guess jeans action: A reason of Guess jeans has been recorded. agent: And the cost of the Guess jeans is 74 right? customer: Correct agent: So your total refund is going to be 174 agent: Is that correct? customer: Yes action: A refund has

In [ ]:
# Shows how many 'member_level' examples contains a value ###Not to keep when pushing to main
contain_df = pd.DataFrame()
contain_df['contain_value'] = final_df['transcript'].str.contains("|".join(['1', 'None', '3', '2']), case=False, na=False)
contain_df.shape
contain_df
contain_df.values.sum()

# label_sets['personal']['member_level']


In [ ]:
label_sets['order']['num_products']

In [ ]:
# ============================================================
# 2️. DEVELOPMENT EVALUATION
# ============================================================
# Purpose: Evaluate your *current best* prompt on development examples.
# Notes:
# - Dev data is unseen during tuning.
# - Run this cell only after training accuracy stabilizes.
# - Checks whether your prompt generalizes well.
# ============================================================

df = build_dataframe(dev_seeded_examples, feature_options, examples)
print("DataFrame shape:", df.shape)

final_df = extraction(df, feature_options)
pd.set_option("display.max_colwidth", 25)
display(final_df.head(250))

accuracy(final_df, feature_options)

In [ ]:
# ============================================================
# 3️. FINAL TEST EVALUATION
# ============================================================
# Purpose: Final evaluation on test data after dev results look good.
# Notes:
# - Use this cell only once prompt tuning is complete.
# - Keep it clean and reproducible.
# ============================================================

df = build_dataframe(train_seeded_examples, feature_options, examples)
print("DataFrame shape:", df.shape)

final_df = extraction(df, feature_options)
pd.set_option("display.max_colwidth", 25)
display(final_df.head(250))

accuracy(final_df, feature_options)
